# 二項分類器

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series

In [ ]:
np.random.seed(20160512)

# t = 0 非感染
n0, mu0, variance0 = 800, [10, 11], 20
data0 = multivariate_normal(mu0, np.eye(2) * variance0, n0)
df0 = DataFrame(data0, columns=['x1', 'x2'])
df0['t'] = 0

# t = 1 感染
n1, mu1, variance1 = 600, [18, 20], 22
data1 = multivariate_normal(mu1, np.eye(2)*variance1, n1)
df1 = DataFrame(data1, columns=['x1', 'x2'])
df1['t'] = 1

# 感染・非感染が混ざった DataFrame
df = pd.concat([df0, df1], ignore_index=True)
df = df.reindex(permutation(df.index)).reset_index(drop=True)

# 80% をトレーニングデータにしてみる
num_data = int(len(df)*0.8)
train_set = df[:num_data]
test_set = df[num_data:]

In [ ]:
train_set.head()

In [ ]:
train_x = train_set[['x1', 'x2']].as_matrix()
train_t = train_set['t'].as_matrix().reshape([len(train_set), 1])
test_x = test_set[['x1', 'x2']].as_matrix()
test_t = test_set['t'].as_matrix().reshape([len(test_set), 1])

## 最尤推定法
X には $x_1$, $x_2$ が入る

In [ ]:
with tf.device('/GPU:0'):
    # '/cpu:0', '/GPU:0'
    x = tf.placeholder(tf.float32, [None, 2])
    w = tf.Variable(tf.zeros([2, 1]))
    w0 = tf.Variable(tf.zeros([1]))
    f = tf.matmul(x, w) + w0
    p = tf.sigmoid(f)
    # 誤差関数
    t = tf.placeholder(tf.float32, [None, 1])
    loss = -tf.reduce_sum(t*tf.log(p) + (1-t)*tf.log(1-p))
    train_step = tf.train.AdamOptimizer().minimize(loss)
    # 正解率
    correct_prediction = tf.equal(tf.sign(p-0.5), tf.sign(t-0.5))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
%%time
# w0_val, w1_val, w2_val = 0.0, 0.0, 0.0
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    i = 0

    train_accuracy = []
    test_accuracy = []
    for _ in range(2500):
        i += 1
        sess.run(train_step, feed_dict={x: train_x, t: train_t})
        acc_val = sess.run(accuracy, feed_dict={x: train_x, t: train_t})
        train_accuracy.append(acc_val)
        acc_val = sess.run(accuracy, feed_dict={x: test_x, t: test_t})
        test_accuracy.append(acc_val)
            
    w0_val, w_val = sess.run([w0, w])
    #global w0_val, w1_val, w2_val
    w0_val, w1_val, w2_val = w0_val[0], w_val[0][0], w_val[1][0]
    
    print ("%f %f %f" % (w0_val, w1_val, w2_val))

In [ ]:
%matplotlib inline
train_set0 = train_set[train_set['t']==0]
train_set1 = train_set[train_set['t']==1]

fig = plt.figure(figsize = (6, 6))
subplot = fig.add_subplot(1, 1, 1)
subplot.set_ylim([0, 30])
subplot.set_xlim([0, 30])
subplot.scatter(train_set1.x1, train_set1.x2, marker='x')
subplot.scatter(train_set0.x1, train_set0.x2, marker='o')

linex = np.linspace(0, 30, 10)
liney = -(w1_val * linex / w2_val + w0_val / w2_val)

subplot.plot(linex, liney)

field = [[(1 / (1 + np.exp(-(w0_val + w1_val * x1 + w2_val * x2))))
            for x1 in np.linspace(0, 30, 100)]
            for x2 in np.linspace(0, 30, 100)]
subplot.imshow(field, origin='lower', extent=(0, 30, 0, 30),
              cmap=plt.cm.gray_r, alpha=0.5)

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
subplot = fig1.add_subplot(1, 1, 1)
subplot.plot(range(len(train_accuracy)), train_accuracy,
            linewidth = 2, label='Training set')
subplot.plot(range(len(test_accuracy)), test_accuracy,
            linewidth = 2, label = 'Test set')
subplot.legend(loc = 'upper left')